In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import random
from glob import glob
from skimage import transform
from tqdm import tqdm
from transformers import AutoTokenizer, BertForMaskedLM
import torch


# ensure is in parent directory
try:
    os.chdir("../../RUG-HandRec/")
except:
    pass

In [1]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
# print vocab
print(tokenizer.vocab_size)
inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


30522


'paris'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

In [ ]:
# feed random hidden states to the model
hidden_states = torch.randn(1, 128, 768)
inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
with torch.no_grad():
    logits = model(inputs.input_ids, hidden_states=hidden_states).logits

In [ ]:
sentence = "The quick brown [MASK] jumps over the lazy dog"
inputs = tokenizer(sentence, return_tensors="pt")
print(inputs)



In [ ]:
print("vocab size: ", len(tokenizer.vocab))
# use inputs_embeds to pass in embeddings for each token
# sample random vector
sequence_length = 16
embedding_size = 768
inputs_embeds = torch.rand(1, sequence_length, embedding_size)

# pay attention to only the first three 
# tokens in the sequence
attention_mask = torch.ones(1, sequence_length)
attention_mask[:, 3:] = 0

print("number of parameters: ", sum(p.numel() for p in model.parameters() if p.requires_grad))

outputs = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
print(outputs.logits.shape)
# get predicted token
predicted_index = torch.argmax(outputs.logits[0, 2]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)


In [ ]:
# load standard bert
from transformers import BertTokenizer, BertModel, BertConfig
# is decoder true
config = BertConfig(is_decoder=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel(config=config).from_pretrained('bert-base-uncased')

vocab_size = len(tokenizer.vocab)
# Feed random input

input_ids = torch.randint(0, vocab_size, (1, 32))
# 0 is padding token, 1000 is vocab size, 16 is sequence length
outputs = model(input_ids)

print(outputs.last_hidden_state.shape)


In [ ]:
import torch
from transformers import BertConfig, BertModel

# Load the pre-trained configuration
config = BertConfig.from_pretrained("bert-base-uncased")

# Set is_decoder and add_cross_attention to True
config.is_decoder = True
config.add_cross_attention = True

# Initialize the model with the modified configuration
decoder_bert = BertModel.from_pretrained("bert-base-uncased", config=config)

# Create random input_ids and attention_mask
batch_size = 2
sequence_length = 10

input_ids = torch.randint(0, config.vocab_size, (batch_size, sequence_length))
attention_mask = torch.ones(batch_size, sequence_length)

# Create random encoder hidden states
encoder_sequence_length = 12
encoder_hidden_states = torch.randn(batch_size, encoder_sequence_length, config.hidden_size)

# Feed the input_ids, attention_mask, and encoder hidden states into the model
output = decoder_bert(input_ids=input_ids, attention_mask=attention_mask, encoder_hidden_states=encoder_hidden_states)

# The output is a BaseModelOutputWithCrossAttentions object


In [ ]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
# feed in random image

image = torch.rand(1, 3, 256, 256)
output = resnet(image)
print(output.shape)
resnet

In [12]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import numpy as np

# feed in ranomd signal
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# random signal 
input_values = processor(np.random.randint(-10, 10, 960), return_tensors="pt").input_values
input_values = input_values.to(torch.float32)
logits = model(input_values).logits
# get text
print(logits.shape)
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])
transcription

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


torch.Size([1, 299, 32])


''